In [1]:
%load_ext autoreload
%autoreload 2

import os
import urllib

from azureml.core import  (Workspace,Run,VERSION,
                           Experiment,Datastore)

from azureml.core.runconfig import (RunConfiguration,
                                    DEFAULT_GPU_IMAGE)
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.compute import (AmlCompute, ComputeTarget)
from azureml.exceptions import ComputeTargetException

from azureml.data.data_reference import DataReference

from azureml.pipeline.core import (Pipeline, 
                                   PipelineData)
from azureml.pipeline.steps import HyperDriveStep
from azureml.pipeline.steps import PythonScriptStep

from azureml.pipeline.steps import HyperDriveStep

from azureml.train.dnn import PyTorch
from azureml.train.hyperdrive import *

from azureml.widgets import RunDetails


print('SDK verison', VERSION)

SDK verison 1.0.17


In [2]:
SUBSCRIPTION_ID = 'fe375bc2-9f1a-4909-ad0d-9319806d5e97'
RESOURCE_GROUP = 'amlenv_rg'
WORKSPACE_NAME = 'vienna'

PROJECT_DIR = os.getcwd()
EXPERIMENT_NAME = "customer_churn"
CLUSTER_NAME = "gpu-cluster"
DATA_DIR = os.path.join(PROJECT_DIR,'data')
SCRIPT_DIR = os.path.join(PROJECT_DIR,'train')

SOURCE_URL ='https://amlgitsamples.blob.core.windows.net/churn'
FILE_NAME = 'CATelcoCustomerChurnTrainingSample.csv'

In [3]:
ws = Workspace(workspace_name = WORKSPACE_NAME,
               subscription_id = SUBSCRIPTION_ID ,
               resource_group = RESOURCE_GROUP)

ws.write_config()

print('Workspace loaded:', ws.name)

Wrote the config file config.json to: /extdrive1/home/sasuke/dev/amlsamples/Customer_churn/aml_config/config.json
Workspace loaded: vienna


In [4]:
os.makedirs(DATA_DIR, exist_ok=True)

urllib.request.urlretrieve(os.path.join(SOURCE_URL,FILE_NAME), 
                           filename = os.path.join(DATA_DIR,FILE_NAME))

('/extdrive1/home/sasuke/dev/amlsamples/Customer_churn/data/CATelcoCustomerChurnTrainingSample.csv',
 <http.client.HTTPMessage at 0x7f7bf3c898d0>)

In [5]:
ds = ws.get_default_datastore()
data_dir = ds.upload(src_dir=DATA_DIR, target_path='churn', overwrite=True, show_progress=True)

Uploading /extdrive1/home/sasuke/dev/amlsamples/Customer_churn/data/CATelcoCustomerChurnTrainingSample.csv
Uploaded /extdrive1/home/sasuke/dev/amlsamples/Customer_churn/data/CATelcoCustomerChurnTrainingSample.csv, 1 files out of an estimated total of 1


In [15]:
cluster_name = "gpu-cluster"

try:
    cluster = ComputeTarget(ws, cluster_name)
    print(cluster_name, "found")
    
except ComputeTargetException:
    print(cluster_name, "not found, provisioning....")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',max_nodes=1)

    
    cluster = ComputeTarget.create(ws, cluster_name, provisioning_config)
    cluster.wait_for_completion(show_output=True)

gpu-cluster found


In [16]:
'''cd = CondaDependencies()
cd.add_conda_package('pandas')
cd.add_conda_package('matplotlib')
cd.add_conda_package('numpy')
cd.add_conda_package('scikit-learn')
#cd.add_channel(channel = 'gpytorch')
cd.add_pip_package('gpytorch')

run_config = RunConfiguration(framework="python",
                              conda_dependencies= cd)
run_config.target = cluster
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_GPU_IMAGE
run_config.environment.python.user_managed_dependencies = False'''

'cd = CondaDependencies()\ncd.add_conda_package(\'pandas\')\ncd.add_conda_package(\'matplotlib\')\ncd.add_conda_package(\'numpy\')\ncd.add_conda_package(\'scikit-learn\')\n#cd.add_channel(channel = \'gpytorch\')\ncd.add_pip_package(\'gpytorch\')\n\nrun_config = RunConfiguration(framework="python",\n                              conda_dependencies= cd)\nrun_config.target = cluster\nrun_config.environment.docker.enabled = True\nrun_config.environment.docker.base_image = DEFAULT_GPU_IMAGE\nrun_config.environment.python.user_managed_dependencies = False'

In [17]:
processed_dir = PipelineData(name = 'processed_outputs')

In [18]:
estimator = PyTorch(source_directory=SCRIPT_DIR,
                    conda_packages = ['pandas', 'numpy', 'scikit-learn'],
                    pip_packages = ['torch>=1.0.0','torchvision','gpytorch'],
                    compute_target=cluster,
                    entry_script='svdkl_entry.py',
                    use_gpu=True)

In [19]:
ps = RandomParameterSampling(
    {
        '--batch-size': choice(512, 1024),
        '--epochs': choice(200),
        '--neural-net-lr': loguniform(-6, -1),
        '--likelihood-lr': loguniform(-6, -1),
        '--grid-size': choice(32,64),
        '--grid_bounds': choice((-1,1),(0,1)),
        '--latent-dim': choice(2),
        '--num-mixtures': choice(2,4,6)
    }
)

early_termination_policy = BanditPolicy(evaluation_interval=10, slack_factor=0.1)

hd_config = HyperDriveRunConfig(estimator=estimator, 
                                hyperparameter_sampling=ps,
                                policy=early_termination_policy,
                                primary_metric_name='auc', 
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                max_total_runs=2,
                                max_concurrent_runs=2)

In [20]:
'''data_ref = DataReference(data_reference_name="data_input",
                         path_on_datastore='churn',
                         datastore=ds,
                         )'''

pre_processing = PythonScriptStep(
                            name = 'preprocess dataset',
                            script_name = 'preprocess.py',
                            arguments = ['--input_path', data_dir,\
                                         '--output_path', processed_dir],
                            inputs = [data_dir],
                            outputs = [processed_dir],
                            compute_target = cluster_name,
                            runconfig = run_config
                        )

In [21]:
hd_step = HyperDriveStep(
    name="hyper parameters tunning",
    hyperdrive_run_config=hd_config,
    estimator_entry_script_arguments=['--data-folder', processed_dir],
    inputs=[processed_dir])

In [22]:
pipeline = Pipeline(workspace=ws, steps=[hd_step])
pipeline_run = Experiment(ws, 'Customer_churn').submit(pipeline)

Created step hyper parameters tunning [6c1d0ae3][c532b622-6013-4e03-9291-d452d662d090], (This step is eligible to reuse a previous run's output)
Created step preprocess dataset [f0c4717d][7539777d-9318-4292-9f63-22e7400258f8], (This step is eligible to reuse a previous run's output)
Using data reference dc399d7cf508496da3b26bde410714c0 for StepId [fa532da6][e6b78b04-c3a5-4837-9e41-08e9196e7813], (Consumers of this data are eligible to reuse prior runs.)
Submitted pipeline run: 05d9e46c-8f9e-4714-9e09-f31fd42c26fa


In [23]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…